In [ ]:
import os
import sys
sys.path.insert(0, "..")
import gc
gc.enable()

In [ ]:
import os
import sys
sys.path.insert(0, os.getcwd())
from utils.DataLoader import  DataLoader
from utils.utilities import *
from utils.visualization import *
from models.fcn8 import FCN8
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

tf.debugging.set_log_device_placement(True)
dl = DataLoader()

In [ ]:
train_df, valid_df = dl.train_split(0.1, filterNan=True)
print("train set shape", train_df.shape)
print("validation set shape", valid_df.shape)

In [ ]:
grouped_images_gen = dl.get_grouped_images_gen(train_df)#, f="adaptive_equalization") # applying filters increases dramatically the execution time
aug_gen = get_augmented_images_generator(grouped_images_gen)
valid_x, valid_y = next(dl.get_grouped_images_gen(valid_df, batch_size=500))#, f="adaptive_equalization")) #valid_df.shape[0]))

In [ ]:
# todo adjust output to classes number

t_x, t_y = next(aug_gen)
x_shape =np.squeeze(t_x[0]).shape
print(x_shape)
fcn_model = FCN8(x_shape, "../models/serialized/")
fcn_model.compile()

In [ ]:
hl = fcn_model.validate(aug_gen, train_df.shape[0], valid_set=(valid_x, valid_y), epochs=15, train_steps=200)
fcn_model.show_loss(hl)

In [ ]:
test_x, test_y = next(dl.get_grouped_images_gen(batch_size=40, train=False))
fcn_model.examine_performance(test_x, test_y, load=True)